In [2]:
from torch.utils.data import Dataset

# Create a toy dataset
class NumberProductDataset(Dataset):
    def __init__(self, data_range=(1, 10)):
        self.numbers = list(range(data_range[0], data_range[1]))

    def __getitem__(self, index):
        number1 = self.numbers[index]
        number2 = self.numbers[index] + 1
        return (number1, number2), number1 * number2

    def __len__(self):
        return len(self.numbers)

In [17]:
from torch.utils.data import DataLoader

# Instantiate the dataset
dataset = NumberProductDataset(data_range=(0, 5))

# Create a DataLoader instance
dataloader = DataLoader(dataset, batch_size=2, shuffle=False)

# Iterating over batches
for (num_pairs, products) in dataloader:
    print(num_pairs, products)
# [tensor([4, 3, 1]), tensor([5, 4, 2])] tensor([20, 12, 2])
# [tensor([2, 0]), tensor([3, 1])] tensor([6, 0])

[tensor([1, 2]), tensor([2, 3])] tensor([2, 6])
[tensor([3, 4]), tensor([4, 5])] tensor([12, 20])
[tensor([5]), tensor([6])] tensor([30])


In [19]:
import torch

torch.randint(0, 10, (64,))

tensor([7, 7, 7, 9, 8, 6, 4, 3, 3, 5, 0, 4, 3, 9, 8, 7, 2, 9, 2, 4, 1, 2, 0, 2,
        8, 6, 1, 5, 3, 1, 7, 8, 7, 4, 6, 8, 2, 1, 6, 0, 9, 2, 9, 6, 7, 7, 6, 3,
        2, 3, 8, 1, 7, 1, 7, 3, 8, 3, 6, 3, 3, 2, 9, 8])